# Getting started with PyProBE

In [ ]:
import pyprobe
from pprint import pprint

%matplotlib inline

## Convert data to standard format

Create the cell object and load some data. If this is the first time that the data has been loaded, it must first be converted into the standard format for PyProBE.

In [ ]:
# Describe the cell. Required fields are 'Name'.
info_dictionary = {
    "Name": "Sample cell",
    "Chemistry": "NMC622",
    "Nominal Capacity [Ah]": 0.04,
    "Cycler number": 1,
    "Channel number": 1,
}

# Create a cell object
cell = pyprobe.Cell(info=info_dictionary)

data_directory = "../../../tests/sample_data/neware"

# Uncomment if running locally
# cell.process_cycler_file(cycler='neware',
#                          folder_path=data_directory,
#                          input_filename='sample_data_neware.xlsx',
#                          output_filename='sample_data_neware.parquet')

If a parquet file exists alongside the original data file, you can add it as a Procedure object to the procedure dictionary of the cell. The key of the dictionary is the procedure name that you provide.

In [ ]:
cell.add_procedure(
    procedure_name="Sample",
    folder_path=data_directory,
    filename="sample_data_neware.parquet",
)

print(cell.procedure)

Importing with this method requires a readme file to sit alongside your data. By default it is called `README.yaml`, but you can specify your own name and pass it as an argument to `add_procedure`. The readme file contains descriptions of the experiments and steps in the procedure:

In [ ]:
import yaml

pprint(yaml.safe_load(open(data_directory + "/README.yaml", "r")))

Once loaded, these can be accessed through the `experiment_names` and `step_descriptions` attributes of the procedure:

In [ ]:
print("Experiment names: ", cell.procedure["Sample"].experiment_names)
print("Step Descriptions: ")
pprint(cell.procedure["Sample"].step_descriptions)

Alternatively, if you need to view data quickly and have not prepared a readme file you can use the `quick_add_procedure` method:

In [ ]:
cell.quick_add_procedure(
    procedure_name="Sample Quick",
    folder_path=data_directory,
    filename="sample_data_neware.parquet",
)

This procedure will have empty `experiment_names` and `step_descriptions` attributes:

In [ ]:
print("Experiment names: ", cell.procedure["Sample Quick"].experiment_names)
print("Step Descriptions: ")
pprint(cell.procedure["Sample Quick"].step_descriptions)

The dashboard can be launched as soon as procedures have been added to the cell (uncomment to run when outside docs environment):

In [ ]:
# pyprobe.launch_dashboard([cell])

The raw data is accessible as a dataframe with the data property:

In [ ]:
print(cell.procedure["Sample"].data)

Individual columns can be returned as 1D numpy arrays with the `get()` method:

In [ ]:
current = (
    cell.procedure["Sample"].experiment("Break-in Cycles").charge(0).get("Current [A]")
)
print(type(current), current)

Multiple columns can be returned at once:

In [ ]:
current, voltage = (
    cell.procedure["Sample"]
    .experiment("Break-in Cycles")
    .charge(0)
    .get("Current [A]", "Voltage [V]")
)
print("Current = ", current)
print("Voltage = ", voltage)

And different unit can be returned on command:

In [ ]:
current_mA = (
    cell.procedure["Sample"].experiment("Break-in Cycles").charge(0).get("Current [mA]")
)
print("Current [mA] = ", current_mA)

Any part of the procedure can be plotted quickly using the ```plot``` method:

In [ ]:
cell.procedure["Sample"].experiment("Break-in Cycles").plot(
    x="Experiment Time [s]", y="Voltage [V]"
)

We can use the `analysis` to further analyse the data. For the `'Break-in Cycles'` we will use the `cycling` analysis module and the functions within. These functions return `Result` objects, so they can be interacted with in the same ways as raw data:

In [ ]:
from pyprobe.analysis import cycling

cycling_summary = cycling.summary(
    input_data=cell.procedure["Sample"].experiment("Break-in Cycles")
)
print(type(cycling_summary))

print(cycling_summary.data)

And it can be plotted as normal too:

In [ ]:
cycling_summary.plot(x="Capacity Throughput [Ah]", y="Discharge Capacity [mAh]")

As the procedure that we imported with the `quick_add_procedure` method does not contain experiment information, that filter will not work on it:

In [ ]:
cycling_summary = cycling.summary(
    input_data=cell.procedure["Sample Quick"].experiment("Break-in Cycles")
)

However, all other filters will still work as expected.